In [ ]:
import ast

In [ ]:
import numpy as np
import scipy as sp
import scipy.stats as sps
import gc
import pandas as pd
import os
from dateutil.parser import parse
from datetime import datetime, timedelta
os.chdir('/Users/lunin-dv/Desktop/Library/')
import importlib
import my_library as lib
import ast
importlib.reload(lib)
os.chdir('/Users/lunin-dv/Desktop/data/Upsell')

In [ ]:
import yt.wrapper as yt
import numpy as np, statsmodels.stats.api as sms
import json

## MARKETO

In [ ]:
def marketo_df(name, table_name):
    path = "//home/cloud_analytics/emailing/experiments/upsell/all_experiments/" + name
    try:
        # Upsell-stream
        req = f"""
        SELECT
            '{table_name}' as table_name,
            billing_account_id,
            billing_account_id as old_billing_account_id,
            email,
            Group,
            all_experiment_size,
            experiment_date
        FROM "{path}" as a,
        (
        SELECT
           count(*) as all_experiment_size 
        FROM "{path}"
        ) as b
        FORMAT TabSeparatedWithNames
        """
        df = lib.execute_query(req)
        return df
    except Exception:
        pass
    
    try:
        # компания на новых клиентов
        req = f"""
        SELECT
            '{table_name}' as table_name,
            old_billing_account_id,
            billing_account_id,
            email,
            'not applicable' as Group,
            all_experiment_size
        FROM "{path}" as a,
        (
        SELECT
           count(*) as all_experiment_size 
        FROM "{path}"
        ) as b
        FORMAT TabSeparatedWithNames
        """
        df = lib.execute_query(req)
        df['old_billing_account_id'] = df['old_billing_account_id'].fillna('')
        return df
    except Exception:
        pass
    
    try:
        # обычная компания без появления новых клиентов
        req = f"""
        SELECT
            '{table_name}' as table_name,
            billing_account_id,
            billing_account_id as old_billing_account_id,
            email,
            Group,
            all_experiment_size
        FROM "{path}" as a,
        (
        SELECT
           count(*) as all_experiment_size 
        FROM "{path}"
        ) as b
        FORMAT TabSeparatedWithNames
        """
        df = lib.execute_query(req)
        return df
    except Exception:
        pass
    
    # обычная компания без появления новых клиентов и без группы
    req = f"""
    SELECT
        '{table_name}' as table_name,
        billing_account_id,
        billing_account_id as old_billing_account_id,
        email,
        'not applicable' as Group,
        all_experiment_size
    FROM "{path}" as a,
    (
        SELECT
           count(*) as all_experiment_size 
        FROM "{path}"
    ) as b
    FORMAT TabSeparatedWithNames
    """
    df = lib.execute_query(req)
    return df

In [ ]:
def update_table(path, table_name):
    # Появился ли биллинг
    full_path = path + '/' + table_name
    req = f"""
    SELECT
        *
    FROM "{full_path}"
    FORMAT TabSeparatedWithNames
    """
    df = lib.execute_query(req)
    if 'billing_account_id' in df.columns:
        df = df.drop(columns = ["billing_account_id"])
    req = """
    SELECT
        DISTINCT
            argMax(billing_account_id, event_time) as billing_account_id,
            user_settings_email as email
    FROM "//home/cloud_analytics/cubes/acquisition_cube/cube"
    GROUP BY user_settings_email
    HAVING billing_account_id != ''
    FORMAT TabSeparatedWithNames
    """
    accs = lib.execute_query(req)
    df = pd.merge(df, accs, on='email', how='left')
    lib.save_table(table_name, path, df)

In [ ]:
def update_table_from_marketo(path, table_name, curr_row):
    # По тем, кто уже был в какой-то компании
    curr_mails = curr_row['mail_name']
    good_mails_req = "mailing_name like '%" +\
    "%'\nOR\nmailing_name like '%".join(list(set(curr_mails))) + "%'"
    mailing_req = f"""
    SELECT
        billing_account_id,
        any(email) as email,
        'not applicable' as Group,
        billing_account_id as old_billing_account_id
    FROM "//home/cloud_analytics_test/cubes/emailing/cube"
    WHERE ({good_mails_req})
    AND isNotNull(delivery_time)
    GROUP BY billing_account_id
    FORMAT TabSeparatedWithNames
    """
    df = lib.execute_query(mailing_req)
    lib.save_table(table_name, 
                   path, df)

In [ ]:
req = """
SELECT
   *
FROM "//home/cloud_analytics/emailing/experiments/upsell/all_experiments/marketo_upsell_tables"
FORMAT TabSeparatedWithNames
"""
int_name_df = lib.execute_query(req)
int_name_df['mail_name'] =\
int_name_df['mail_name'].apply(lambda x: ast.literal_eval(x.replace("\\", "")))

In [ ]:
path = "//home/cloud_analytics/emailing/experiments/upsell/all_experiments"
for table_name in int_name_df[int_name_df['need_to_update_before_starting'] == 1]['path']:
    update_table(path, table_name)

In [ ]:
path = "//home/cloud_analytics/emailing/experiments/upsell/all_experiments"
for table_name in int_name_df[int_name_df['need_to_update_before_starting'] == 2]['path']:
    curr_row = int_name_df[int_name_df['path'] == table_name].iloc[0]
    update_table_from_marketo(path, curr_row['path'], curr_row)

In [ ]:
all_mails = []
for array in int_name_df['mail_name']:
    all_mails += array

In [ ]:
good_mails_req = "mailing_name like '%" +\
    "%'\nOR\nmailing_name like '%".join(list(set(all_mails))) + "%'"
mailing_req = f"""
SELECT
    billing_account_id,
    mailing_name,
    if (isNotNull(max(open_time)), 1, 0) as open_letter,
    if (isNotNull(max(click_time)), 1, 0) as click_letter,
    max(delivery_time) as experiment_date
FROM "//home/cloud_analytics_test/cubes/emailing/cube"
WHERE ({good_mails_req})
AND isNotNull(delivery_time)
GROUP BY billing_account_id, mailing_name
HAVING isNotNull(experiment_date)
FORMAT TabSeparatedWithNames
"""

mail_df = lib.execute_query(mailing_req)

In [ ]:
def is_good_mail(x, curr_names):
    for name in curr_names:
        if not pd.isnull(x) and name in x:
            return True
    return False

In [ ]:
def group_mail_df_by_many_mails_into_one(curr_mail_df):
    grouped = curr_mail_df.groupby("billing_account_id")
    res = grouped.apply(lambda x: x.sort_values(['experiment_date'])\
              .sort_values(['open_letter'], ascending=False).head(1))
    res['billing_account_id'] = np.array(res.index.to_list()).T[0]
    res.index = np.arange(0, len(res))
    return res

In [ ]:
dfs = []
for _, row in int_name_df.iterrows():
    curr_df = marketo_df(row['path'], row['table_name'])
    curr_df = curr_df[~curr_df['billing_account_id'].isna()]
    curr_names = row['mail_name']
    curr_mail_df = mail_df[mail_df["mailing_name"].apply(lambda x: is_good_mail(x, 
                                                                                curr_names))]
    curr_mail_df = group_mail_df_by_many_mails_into_one(curr_mail_df)
    # test group
    test_df = curr_df[(curr_df['Group'] == 'test') |
                           (curr_df['Group'] == 'not applicable')]
    if 'experiment_date' in test_df.columns:
        test_df = test_df.drop(columns=['experiment_date'])
    test_df = pd.merge(test_df,curr_mail_df[['billing_account_id', 'experiment_date', 
                                             'open_letter', 'click_letter']], 
                       on = 'billing_account_id', how='inner')
    
    #date
    test_df['experiment_date'] = test_df['experiment_date'].apply(
        lambda x: lib.date_to_string(parse(x)))
    control_date = test_df['experiment_date'].min()
    columns = ["table_name", 'old_billing_account_id',
               "billing_account_id", "Group", "experiment_date", 
               "open_letter", "click_letter", 'all_experiment_size']
    test_df = test_df[columns]

    final_table = test_df
    
    if 'control' in curr_df["Group"].unique():
        control_df = curr_df[curr_df["Group"] == 'control']
        if 'experiment_date' not in control_df.columns:
            control_df.loc[:, 'experiment_date'] = [control_date] * len(control_df)
        
        control_df.loc[:, 'open_letter'] = [0] * len(control_df)
        control_df.loc[:, 'click_letter'] = [0] * len(control_df)
        control_df = control_df[columns]
        final_table = lib.concatenate_tables([test_df, control_df])
    
    dfs.append(final_table)
    print(row['table_name'], test_df.shape, final_table.shape)

In [ ]:
marketo_main_df = lib.concatenate_tables(dfs)

# Рассылятор

In [ ]:
def is_good_mail(x, curr_names, num_to_check):
    cnt = 0
    for name in curr_names:
        if name in x:
            cnt += 1
    if cnt >= num_to_check:
        return True
    return False

In [ ]:
req = """
SELECT
   *
FROM "//home/cloud_analytics/emailing/sender/upsell_tables"
FORMAT TabSeparatedWithNames
"""
int_name_df = lib.execute_query(req)
int_name_df['mail_name'] =\
int_name_df['mail_name'].apply(lambda x: ast.literal_eval(x.replace("\\", "")))

In [ ]:
tags = list(int_name_df['mail_name'])
tags =  set([item for sublist in tags for item in sublist])

good_mails_req = "tags == '" +\
    "'\nOR\n tags == '".join(list(set(tags))) + "'"

In [ ]:
def get_mails():
    req = f"""
    SELECT
        DISTINCT
            email,
            tags,
            event,
            unixtime,
            toDate(unixtime / 1000) as send_time,
            halfMD5(email || '_' || message_id) as key
    FROM "//home/cloud_analytics/import/emails/emails_delivery_clicks"
    WHERE source == 'sender'
    and event in ('click', 'px', 'send')
    and ({good_mails_req})
    FORMAT TabSeparatedWithNames
    """
    mail_df = lib.execute_query(req)
    mail_df['key'] = mail_df['key'].astype(str)
    lib.save_table("emailing_upsell_tmp", '//home/cloud_analytics/lunin-dv/tmp', mail_df)
    
    mail_df = lib.execute_query("""
    SELECT
        email,
        experiment_date,
        open_letter,
        click_letter,
        mailing_name
    FROM (
        SELECT
            email,
            max(if (event == 'send', send_time, null)) as experiment_date,
            max(if (event == 'px', 1, 0)) as open_letter,
            max(if (event == 'click', 1, 0)) as click_letter,
            groupUniqArray(tags) as mailing_name
        FROM "//home/cloud_analytics/lunin-dv/tmp/emailing_upsell_tmp"
        GROUP BY email, key
        HAVING toDate(experiment_date) > toDate('2020-01-01')
    ) as sender
    FORMAT TabSeparatedWithNames
    """)
    return mail_df

In [ ]:
mail_df = get_mails()

In [ ]:
mail_df['open_letter'].sum()

In [ ]:
def continuous_df(name, table_name):
    path = "//home/cloud_analytics/emailing/sender/"
    try:
        req = f"""
        SELECT
            email,
            billing_account_id,
            billing_account_id as old_billing_account_id,
            time as experiment_date,
            group as Group,
            '{table_name}' as table_name,
            all_experiment_size
        FROM "{path + name}" as a,
        (
            SELECT
               count(*) as all_experiment_size 
            FROM "{path + name}"
        ) as b
        FORMAT TabSeparatedWithNames
        """
        df = lib.execute_query(req)
    except Exception:
        req = f"""
        SELECT
            email,
            billing_account_id,
            billing_account_id as old_billing_account_id,
            experiment_date,
            Group,
            '{table_name}' as table_name,
            all_experiment_size
        FROM "{path + name}" as a,
        (
            SELECT
               count(*) as all_experiment_size 
            FROM "{path + name}"
        ) as b
        FORMAT TabSeparatedWithNames
        """
        df = lib.execute_query(req)
    return df

In [ ]:
def group_mail_df_by_many_mails_into_one(curr_mail_df):
    grouped = curr_mail_df.groupby("email")
    res = grouped.apply(lambda x: x.sort_values(['experiment_date']).head(1))

    col = 'open_letter'
    res = pd.merge(res.drop(columns =[col]), 
         pd.DataFrame(grouped.apply(lambda x: x[col].max()), columns=[col]),
         left_index=True, right_index=True)
    
    col = 'click_letter'
    res = pd.merge(res.drop(columns =[col]), 
         pd.DataFrame(grouped.apply(lambda x: x[col].max()), columns=[col]),
         left_index=True, right_index=True)
    
    res['email'] = np.array(res.index.to_list()).T[0]
    res.index = np.arange(0, len(res))
    return res

In [ ]:
dfs = []
for _, row in int_name_df.iterrows():
    curr_df = continuous_df(row['path'], row['table_name'])
    curr_names = row['mail_name']
    num_to_check = row['min_num_of_common_tags']
    curr_mail_df = mail_df[mail_df["mailing_name"].apply(lambda x: is_good_mail(x, 
                                                                                curr_names,
                                                                                num_to_check))]
    curr_mail_df = group_mail_df_by_many_mails_into_one(curr_mail_df)
    # test group
    test_df = pd.merge(curr_df[(curr_df['Group'] == 'test') |
                           (curr_df['Group'] == 'not applicable')].drop(columns=['experiment_date']), 
                    curr_mail_df[
                        ['email', 'experiment_date', 'open_letter', 'click_letter']
                    ], 
                   on = 'email', how='inner')
    columns = ["table_name", 'old_billing_account_id',
           "billing_account_id", "Group", "experiment_date", 
           "open_letter", "click_letter", 'all_experiment_size']
    test_df = test_df[columns]

    
    final_table = test_df
    if 'control' in curr_df["Group"].unique():
        control_df = curr_df[curr_df["Group"] == 'control']
        control_df.loc[:, 'open_letter'] = [0] * len(control_df)
        control_df.loc[:, 'click_letter'] = [0] * len(control_df)
        control_df = control_df[columns]
        final_table = lib.concatenate_tables([test_df, control_df])
    dfs.append(final_table)

In [ ]:
continious_df = lib.concatenate_tables(dfs)

In [ ]:
main_df = lib.concatenate_tables([marketo_main_df, continious_df])

# ОСНОВНОЕ 

## Гранты (старый вариант)

In [ ]:
promos_request = f"""
SELECT
    DISTINCT billing_account_id,
    source_id,
    id as activate_id
FROM "//home/cloud/billing/exported-billing-tables/monetary_grants_prod"
FORMAT TabSeparatedWithNames
"""
promos_df = lib.execute_query(promos_request)
upsell_codes_req = """
SELECT
    *
FROM "//home/cloud_analytics/emailing/experiments/upsell/grant_sources/upsell_grants"
FORMAT TabSeparatedWithNames
"""
upsell_codes_df = lib.execute_query(upsell_codes_req)

activated_codes_df = pd.merge(upsell_codes_df, promos_df, on='source_id')

## Гранты автоматический вариант

In [ ]:
activated_codes_new = lib.execute_query("""
SELECT
    *
FROM (
SELECT
    DISTINCT
        source_id,
        splitByChar(' ',  assumeNotNull(upsell_experiment_names)) as table_name,
        billing_account_id,
        id as activate_id
FROM "//home/cloud_analytics/lunin-dv/grants/offers_grants_information_table"
WHERE isNotNull(upsell_experiment_names)
AND isNotNull(billing_account_id)
AND upsell_experiment_names != '-'
)
ARRAY JOIN table_name
FORMAT TabSeparatedWithNames
""")


In [ ]:
activated_codes_df = lib.concatenate_tables([activated_codes_df, activated_codes_new])

In [ ]:
activated_codes_df.drop_duplicates(inplace=True)

In [ ]:
new_tables = []
for name in main_df["table_name"].unique():
    curr_table = main_df[main_df["table_name"] == name].copy()
    curr_activated_codes_df = activated_codes_df[activated_codes_df["table_name"] == name]
    if len(curr_activated_codes_df) > 0:
        activates = np.array(curr_table["billing_account_id"].isin(\
            curr_activated_codes_df["billing_account_id"]).astype(int))
        curr_table["activate_grant"] = activates
        curr_table = pd.merge(curr_table, curr_activated_codes_df,
                              on=['billing_account_id', 'table_name'], how='left')
        curr_table['activate_id'] = curr_table['activate_id'].fillna('')
    else:
        curr_table["activate_grant"] = [-1] * len(curr_table)
        curr_table["source_id"] = ''
        curr_table["activate_id"] = ''
    new_tables.append(curr_table)
    
df_with_grants = lib.concatenate_tables(new_tables)

In [ ]:
df_with_grants = df_with_grants.fillna('')

In [ ]:
assert main_df.shape[0] == df_with_grants.shape[0]

In [ ]:
import time

In [ ]:
def add_vm_new_services(table_raw, day_interval):
    table = table_raw.copy()
    lib.save_table("emailing_upsell_tmp", '//home/cloud_analytics/lunin-dv/tmp', 
                   table)
    time.sleep(20)
    upper_bound = ""
    if day_interval != -1:
        upper_bound = f"""toDate(vm_start) <= addDays(toDate(experiment_date), {day_interval})
                          AND toDate(vm_start) >= toDate(experiment_date)"""
    else:
        upper_bound = f"toDate(vm_start) >= toDate(experiment_date)"
    lower_bound = f"""toDate(vm_start) >= addDays(toDate(experiment_date), -60) AND
                    toDate(vm_start) < toDate(experiment_date)"""
    name_add = day_interval
    if name_add == -1:
        name_add = "all"
    req = f"""
    SELECT
        ba_id as billing_account_id,
        table_name,
        count(DISTINCT if ({lower_bound}, node_az, null)) as old_az_num,
        count(DISTINCT if ({upper_bound}, node_az, null)) as new_az_num,
        if (old_az_num == 1 and new_az_num > 1, 1, 0) as 
        start_new_service_az_days_{name_add}
    FROM "//home/cloud_analytics/compute_logs/vm_cube/vm_cube" as a
    INNER JOIN (
        SELECT
            *
        FROM "//home/cloud_analytics/lunin-dv/tmp/emailing_upsell_tmp"
    ) as b
    on a.ba_id == b.billing_account_id
    GROUP BY billing_account_id, table_name
    FORMAT TabSeparatedWithNames
"""
    df = lib.execute_query(req)
    vm_services_df = df[['billing_account_id', 'table_name', 
                                     f'start_new_service_az_days_{name_add}']]
    table = pd.merge(table, vm_services_df, 
                     on = ['billing_account_id', 'table_name'], how='left')
    table = table.fillna(0)
    return table, [f'start_new_service_az_days_{name_add}']

In [ ]:
def add_cube_new_services(table_raw, day_interval, cube_services):
    table = table_raw.copy()
    lib.save_table("emailing_upsell_tmp", '//home/cloud_analytics/lunin-dv/tmp', 
                   table)
    time.sleep(20)
    after_cube_req = ""
    before_cube_req = ""
    diff_cube_req = ""
    upper_bound_after = ""
    if day_interval != -1:
        upper_bound = f"""toDate(event_time) <= addDays(toDate(experiment_date), 
                                                {day_interval})
                          AND toDate(event_time) >= toDate(experiment_date)"""
    else:
        upper_bound = f"toDate(event_time) >= toDate(experiment_date)"
    lower_bound = f"""toDate(event_time) >= addDays(toDate(experiment_date), -60) AND
                    toDate(event_time) < toDate(experiment_date)"""
    name_add = day_interval
    if name_add == -1:
        name_add = "all"

    for service, name in cube_services:
        after_service_str = f"max(if (sku_name like '%{service}%' "\
                            f"and {upper_bound}, 1, 0)) as after_{name},\n"

        before_service_str = f"max(if (sku_name like '%{service}%'"\
                     f"and {lower_bound}, 1, 0)) as before_{name},\n"
        
        diff_service_str = f"if (before_{name} == 0 and after_{name} == 1, 1, 0) "\
                           f"as start_new_service_{name}_days_{name_add},\n"
        diff_cube_req += diff_service_str
        after_cube_req += after_service_str
        before_cube_req += before_service_str

    req = f"""
        SELECT
            {after_cube_req}
            {before_cube_req}
            {diff_cube_req}
            billing_account_id,
            table_name
        FROM "//home/cloud_analytics/cubes/acquisition_cube/cube" as a
        INNER JOIN (
            SELECT
                *
            FROM "//home/cloud_analytics/lunin-dv/tmp/emailing_upsell_tmp"
        ) as b
        ON a.billing_account_id == b.billing_account_id
        GROUP BY billing_account_id, table_name
        FORMAT TabSeparatedWithNames
    """
    df = lib.execute_query(req)

    interest_cols = [f"start_new_service_{name}_days_{name_add}" 
                     for service, name in cube_services]
    
    services_df = df[interest_cols + ["billing_account_id", 'table_name']]
    table = pd.merge(table, services_df, on=["billing_account_id", 'table_name'], 
                     how='left')
    table = table.fillna(0)
    return table, interest_cols

In [ ]:
cube_services = [('support', 'support'), 
                 ('ms.sql', 'sql_server'), 
                 ('marketplace', 'marketplace'), 
                 ('windows', 'windows'),
                 ("mdb", 'mdb'),
                 ("compute", 'compute'),
                 ('mk8s', 'mk8s'), 
                 ('mongo', 'mongo'), 
                 ('pg', 'pg'),
                 ('clickhouse', 'clickhouse'), 
                 ('redis', 'redis'), 
                 ('mysql', 'mysql'),
                 ('hystax', 'hystax')]

In [ ]:
days_to_observe = [14, 28, 63, -1]

In [ ]:
service_cols = []
full_service_compare_df = df_with_grants.copy()
for day_interval in days_to_observe:
    
    full_service_compare_df, service_cols_curr_1 = \
        add_cube_new_services(full_service_compare_df, day_interval, cube_services)

    full_service_compare_df, service_cols_curr_2 = \
        add_vm_new_services(full_service_compare_df, day_interval)
    
    service_cols_curr = service_cols_curr_1 + service_cols_curr_2
    
    
    name_add = day_interval
    if name_add == -1:
        name_add = "all"
    full_service_compare_df[f'start_new_service_all_days_{name_add}'] =\
    full_service_compare_df[service_cols_curr].apply(
        lambda x: 1 if x.sum() > 0 else 0, axis=1)
    service_cols_curr += [f'start_new_service_all_days_{name_add}']
    service_cols += service_cols_curr

In [ ]:
assert full_service_compare_df.shape[0] == main_df.shape[0]

In [ ]:
full_service_compare_df["days_to_now"] = (parse(lib.get_current_date_as_str()) -\
                                          pd.to_datetime(
                                            full_service_compare_df["experiment_date"])
                                          ).dt.days

In [ ]:
full_service_compare_df["days_to_now"] =\
(full_service_compare_df["days_to_now"] // 7) * 7 

full_service_compare_df["days_to_now"] = full_service_compare_df["days_to_now"].apply(
    lambda x: x if x >= 0 else 0)

full_service_compare_df["real_days_to_now"] = full_service_compare_df["days_to_now"].copy()

full_service_compare_df["days_to_now"] = full_service_compare_df["days_to_now"].apply(
    lambda x: x if x < 63 else 63)


In [ ]:
def add_cons_df(table, day_interval, cube_services):
    lib.save_table("emailing_upsell_tmp", '//home/cloud_analytics/lunin-dv/tmp', 
                   table)
    time.sleep(20)
    name_add = day_interval
    if name_add == -1:
        name_add = "all"
     
    if day_interval > 63:
        day_interval = 63

    after_interval = f"""toDate(event_time) < addDays(toDate(experiment_date), 
                                                       to_observe)
                  AND toDate(event_time) >= toDate(experiment_date)"""
    before_interval = f"""toDate(event_time) >= addDays(toDate(experiment_date), 
                                                -to_observe)
                      AND toDate(event_time) < toDate(experiment_date)"""
    
    after_cons_req = ""
    before_cons_req = ""
    diff_req = ""
    for service, name in cube_services:
        after_cons_req += f"""
        sum(if({after_interval} and sku_name like '%{service}%', 
               real_consumption, 0)) as 
        after_consumption_{name}_{name_add},\n
        """
        
        before_cons_req += f"""
        sum(if({before_interval} and sku_name like '%{service}%', 
               real_consumption, 0)) 
        as before_consumption_{name}_{name_add},\n
        """
        
        diff_req += f"""
        (after_consumption_{name}_{name_add} - 
        before_consumption_{name}_{name_add}) / to_observe * 30
        as difference_{name}_{name_add},\n
        """
    
    after_cons_req += f"""
        sum(if({after_interval}, 
               real_consumption, 0)) as 
        after_consumption_all_{name_add},\n
        """

    before_cons_req += f"""
        sum(if({before_interval}, 
            real_consumption, 0)) 
        as before_consumption_all_{name_add},\n
        """
        
    diff_req += f"""
    (after_consumption_all_{name_add} - 
    before_consumption_all_{name_add}) / to_observe * 30
    as difference_all_{name_add},\n
    """
    
    cons_req = f"""
    SELECT
        {after_cons_req}
        {before_cons_req}
        {diff_req}
        billing_account_id,
        table_name,
        experiment_date 
    FROM "//home/cloud_analytics/cubes/acquisition_cube/cube" as a
    INNER JOIN (
        SELECT
            *,
            if (assumeNotNull(days_to_now) > {day_interval} and {day_interval} != -1, {day_interval}, 
                days_to_now) as to_observe
        FROM "//home/cloud_analytics/lunin-dv/tmp/emailing_upsell_tmp"
    ) as b
    ON a.billing_account_id == b.billing_account_id
    GROUP BY billing_account_id, to_observe, table_name, experiment_date
    FORMAT TabSeparatedWithNames
    """
    cons_df = lib.execute_query(cons_req)
    cons_df = cons_df[cons_df["billing_account_id"].isin(table["billing_account_id"])]

    consumption_compare_df = pd.merge(table, 
                                      cons_df, 
                                      on=["billing_account_id", 'table_name', 
                                          'experiment_date'], how="left")
    consumption_compare_df.replace(np.nan, 0, inplace=True)
    return consumption_compare_df

In [ ]:
def add_revenue_for_service_group(consumption_compare_df, day_interval, 
                                  service_name, name_add):
    diff = f"difference_{service_name}_{name_add}"
    if "control" in consumption_compare_df["Group"].unique():

        control_diff = consumption_compare_df[
            (consumption_compare_df["Group"] == 'control') &
            (consumption_compare_df["days_to_now"] >= day_interval)
        ][diff]

        test_diff = consumption_compare_df[
            (consumption_compare_df["Group"] == 'test') &
            (consumption_compare_df["days_to_now"] >= day_interval)
        ][diff]
        
        control_avg = np.mean(np.array(control_diff))
        test_avg = np.mean(np.array(test_diff))
        
        consumption_compare_df[f"experiment_revenue_{service_name}_{name_add}"] =\
        [test_avg - control_avg] * len(consumption_compare_df)
        
        
        ## ttest
        test = test_diff
        control = control_diff
#         print(consumption_compare_df['table_name'].iloc[0], day_interval)
#         print('mean:', np.mean(test), "; std:", np.std(test), '-- TEST')
#         print('mean:', np.mean(control), "; std:", np.std(control), '-- CONTROL')
        pval = sps.ttest_ind(control, test, equal_var=False).pvalue
        
        consumption_compare_df[f"revenue_{service_name}_{name_add}_pval"] = pval

        cm = sms.CompareMeans(sms.DescrStatsW(test), 
                              sms.DescrStatsW(control))
        left, right = cm.tconfint_diff(usevar='unequal')
        consumption_compare_df[f"minimum_revenue_{service_name}_{name_add}"] = left
        consumption_compare_df[f"maximum_revenue_{service_name}_{name_add}"] = right
    else:
        consumption_compare_df[f"experiment_revenue_{service_name}_{name_add}"] = -1
        consumption_compare_df[f"revenue_{service_name}_{name_add}_pval"] = -1
        consumption_compare_df[f"minimum_revenue_{service_name}_{name_add}"] = -1
        consumption_compare_df[f"maximum_revenue_{service_name}_{name_add}"] = -1

    return consumption_compare_df

In [ ]:
def add_revenue_for_group(consumption_compare_df, day_interval, cube_services):
    name_add = day_interval
    if name_add == -1:
        name_add = "all"
    if day_interval == -1:
        day_interval = 7
    
    for _, service_name in cube_services:
        consumption_compare_df =\
        add_revenue_for_service_group(consumption_compare_df, day_interval, 
                                      service_name, name_add)
    
    return consumption_compare_df

In [ ]:
def add_by_day_intervals_cons_df(table, days_to_observe, cube_services):
    consumption_compare_df = table.copy()
    for day_interval in days_to_observe:
        new_tables = []
        consumption_compare_df = add_cons_df(consumption_compare_df, 
                                             day_interval,
                                             cube_services)
        for name in consumption_compare_df["table_name"].unique():
            curr_table = consumption_compare_df[
                    (consumption_compare_df["table_name"] == name)].copy()
            new_tables.append(add_revenue_for_group(curr_table, 
                                                    day_interval, 
                                                    cube_services + [('all', 'all')]))
        consumption_compare_df = lib.concatenate_tables(new_tables)
    return consumption_compare_df

In [ ]:
full_consumption_service_compare_df= add_by_day_intervals_cons_df(
    full_service_compare_df, days_to_observe, [])

In [ ]:
assert main_df.shape[0] == full_consumption_service_compare_df.shape[0]

In [ ]:
def wald_comparison(curr_table, service_cols):
    for service in sorted(service_cols):
        test_df = curr_table[(curr_table["Group"] == 'test') &
                             (curr_table[service] != -1)]
        control_df = curr_table[(curr_table["Group"] == 'control') &
                                (curr_table[service] != -1)]
        if len(test_df) == 0:
            curr_table[service + "_pval"] = [-1 for _ in range(len(curr_table))]
            continue
        test = np.array(test_df[service])
        control = np.array(control_df[service])
        pval = lib.WaldTest(len(control), np.sum(control),
                        len(test), np.sum(test), alternative="less").pvalue
        curr_table[service + "_pval"] = [pval for _ in range(len(curr_table))]
    return curr_table

In [ ]:
new_tables = []
for name in full_consumption_service_compare_df["table_name"].unique():
    #print(name)
    curr_table = full_consumption_service_compare_df[
        (full_consumption_service_compare_df["table_name"] == name)
    ].copy()
    if "control" in curr_table["Group"].unique():
        curr_table = wald_comparison(curr_table, service_cols)
    else:
        for service in sorted(service_cols):
            curr_table[service + "_pval"] = [0.5 for _ in range(len(curr_table))]
    new_tables.append(curr_table)

In [ ]:
result_table = lib.concatenate_tables(new_tables)

In [ ]:
result_table["experiment_name"] = result_table["table_name"]

In [ ]:
result_table.replace(np.nan, 0, inplace=True)

In [ ]:
lib.save_table("upsell_emailing", 
               "//home/cloud_analytics/emailing/experiments/upsell/results", 
               result_table)

In [ ]:
lib.save_table_from_yt_to_grafana(
    "//home/cloud_analytics/emailing/experiments/upsell/results/upsell_emailing",
    "cloud_analytics.upsell_emailing")